<a href="https://colab.research.google.com/github/tul17ii/Mini-Fitness-Function-Project/blob/main/ffmf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

# Step 1: Define possible values for genes
crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Step 2: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    return random.uniform(0, 50)

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 3: Generate random chromosome with added soil nutrients, farm size, and farm type
def generate_random_chromosome():
    # Limit fertilizer selection based on farm size
    farm_size = random.choice(farm_sizes)
    if farm_size == "Small":
        available_fertilizers = ["Organic", "Nitrogen"]
    elif farm_size == "Medium":
        available_fertilizers = fertilizers
    else:  # Large farms have all fertilizers
        available_fertilizers = fertilizers

    fertilizer = random.choice(available_fertilizers)

    # Limit pesticide selection based on farm type
    farm_type = random.choice(farm_types)
    if farm_type == "Organic":
        available_pesticides = ["None"]
    else:  # Conventional farms can use any pesticide
        available_pesticides = pesticides

    pesticide = random.choice(available_pesticides)

    # Limit soil nutrient selection based on farm size
    soil_nutrient = random.choice(soil_nutrients)

    return [
        random.choice(crop_types),
        random.choice(planting_times),
        fertilizer,
        pesticide,
        soil_nutrient,
        farm_size,
        farm_type
    ]

# Step 4: Initialize population
population_size = 10
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 5: Parent selection using fitness-based roulette selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)

    # Normalize fitness values to sum to 1
    normalized_fitness = [fitness / total_fitness for fitness in fitness_values]

    # Select two parents based on fitness probabilities
    parent1 = random.choices(population, weights=normalized_fitness, k=1)[0]
    parent2 = random.choices(population, weights=normalized_fitness, k=1)[0]

    return parent1, parent2

# Step 6: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 7: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        chromosome[mutation_point] = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        chromosome[mutation_point] = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        chromosome[mutation_point] = random.choice(farm_sizes)
    else:  # Farm type mutation
        chromosome[mutation_point] = random.choice(farm_types)

# Step 8: Mutation with temperature
def mutate_with_temperature(chromosome, temperature):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        new_value = random.choice(crop_types)
    elif mutation_point == 1:
        new_value = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        new_value = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        new_value = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        new_value = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        new_value = random.choice(farm_sizes)
    else:  # Farm type mutation
        new_value = random.choice(farm_types)

    # Simulated annealing-inspired acceptance
    original_fitness = combined_fitness(chromosome)
    chromosome[mutation_point] = new_value
    new_fitness = combined_fitness(chromosome)

    if new_fitness < original_fitness:
        acceptance_probability = random.uniform(0, 1)
        if acceptance_probability > (1 / (1 + temperature)):  # Revert mutation with a probability
            chromosome[mutation_point] = new_value

    return chromosome

# Step 9: User input for fitness function selection
print("Select a fitness function:")
print("1. Crop Yield Fitness")
print("2. Environmental Impact Fitness")
print("3. Combined Fitness")

user_choice = int(input("Enter 1, 2, or 3: "))

if user_choice == 1:
    fitness_function = crop_yield_fitness
elif user_choice == 2:
    fitness_function = environmental_impact_fitness
elif user_choice == 3:
    fitness_function = combined_fitness
else:
    print("Invalid choice! Defaulting to Crop Yield Fitness.")
    fitness_function = crop_yield_fitness

# Step 10: Gather user inputs
print("\nPlease answer the following questions:")

fertilizer_pesticide_info = {
    "fertilizers": input("What fertilizers and pesticides do you currently have access to? ").split(", "),
    "costs": input("What is the cost of each fertilizer and pesticide you use? ").split(", "),
    "environmental_impact": input("Do you know the environmental impact or eco-scores of these products? (yes/no) "),
    "nutrient_requirements": input("What are the nutrient requirements for your soil or crops? ").split(", "),
    "farm_size": input("What is the size of your farm (in acres or hectares)? ")
}

print("\nGathered inputs:", fertilizer_pesticide_info)

# Step 11: Genetic Algorithm with the selected fitness function
best_ga_solution, best_ga_fitness = genetic_algorithm_with_temperature(fitness_function)
print(f"\nBest solution using Genetic Algorithm: {best_ga_solution} with Fitness: {best_ga_fitness}")


Select a fitness function:
1. Crop Yield Fitness
2. Environmental Impact Fitness
3. Combined Fitness
Enter 1, 2, or 3: 2

Please answer the following questions:
What fertilizers and pesticides do you currently have access to? nitrogen
What is the cost of each fertilizer and pesticide you use? 50
Do you know the environmental impact or eco-scores of these products? (yes/no) no
What are the nutrient requirements for your soil or crops? soil
What is the size of your farm (in acres or hectares)? 10

Gathered inputs: {'fertilizers': ['nitrogen'], 'costs': ['50'], 'environmental_impact': 'no', 'nutrient_requirements': ['soil'], 'farm_size': '10'}


NameError: name 'genetic_algorithm_with_temperature' is not defined

In [ ]:
import random

# Step 1: Define possible values for genes
crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Additional Inputs
fertilizer_costs = {"Nitrogen": 20, "Phosphorus": 25, "Potassium": 30, "Organic": 15}
pesticide_costs = {"Herbicide": 18, "Insecticide": 22, "Fungicide": 24, "None": 0}

fertilizer_impact = {"Nitrogen": 40, "Phosphorus": 30, "Potassium": 25, "Organic": 10}  # Eco-impact or toxicity score
pesticide_impact = {"Herbicide": 50, "Insecticide": 45, "Fungicide": 35, "None": 0}  # Eco-impact or toxicity score

# Step 2: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    # Summing up the impact of the selected fertilizer and pesticide
    impact = fertilizer_impact[chromosome[2]] + pesticide_impact[chromosome[3]]
    return impact

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 3: Generate random chromosome with added soil nutrients, farm size, and farm type
def generate_random_chromosome():
    # Limit fertilizer selection based on farm size
    farm_size = random.choice(farm_sizes)
    if farm_size == "Small":
        available_fertilizers = ["Organic", "Nitrogen"]
    elif farm_size == "Medium":
        available_fertilizers = fertilizers
    else:  # Large farms have all fertilizers
        available_fertilizers = fertilizers

    fertilizer = random.choice(available_fertilizers)

    # Limit pesticide selection based on farm type
    farm_type = random.choice(farm_types)
    if farm_type == "Organic":
        available_pesticides = ["None"]
    else:  # Conventional farms can use any pesticide
        available_pesticides = pesticides

    pesticide = random.choice(available_pesticides)

    # Limit soil nutrient selection based on farm size
    soil_nutrient = random.choice(soil_nutrients)

    return [
        random.choice(crop_types),
        random.choice(planting_times),
        fertilizer,
        pesticide,
        soil_nutrient,
        farm_size,
        farm_type
    ]

# Step 4: Initialize population
population_size = 10
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 5: Parent selection using fitness-based roulette selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)

    # Normalize fitness values to sum to 1
    normalized_fitness = [fitness / total_fitness for fitness in fitness_values]

    # Select two parents based on fitness probabilities
    parent1 = random.choices(population, weights=normalized_fitness, k=1)[0]
    parent2 = random.choices(population, weights=normalized_fitness, k=1)[0]

    return parent1, parent2

# Step 6: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 7: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        chromosome[mutation_point] = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        chromosome[mutation_point] = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        chromosome[mutation_point] = random.choice(farm_sizes)
    else:  # Farm type mutation
        chromosome[mutation_point] = random.choice(farm_types)

# Step 8: Mutation with temperature
def mutate_with_temperature(chromosome, temperature):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        new_value = random.choice(crop_types)
    elif mutation_point == 1:
        new_value = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        new_value = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        new_value = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        new_value = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        new_value = random.choice(farm_sizes)
    else:  # Farm type mutation
        new_value = random.choice(farm_types)

    # Simulated annealing-inspired acceptance
    original_fitness = combined_fitness(chromosome)
    chromosome[mutation_point] = new_value
    new_fitness = combined_fitness(chromosome)

    if new_fitness < original_fitness:
        acceptance_probability = random.uniform(0, 1)
        if acceptance_probability > (1 / (1 + temperature)):  # Revert mutation with a probability
            chromosome[mutation_point] = new_value

    return chromosome

# Step 9: User input for fitness function selection
print("Select a fitness function:")
print("1. Crop Yield Fitness")
print("2. Environmental Impact Fitness")
print("3. Combined Fitness")

user_choice = int(input("Enter 1, 2, or 3: "))

if user_choice == 1:
    fitness_function = crop_yield_fitness
elif user_choice == 2:
    fitness_function = environmental_impact_fitness
elif user_choice == 3:
    fitness_function = combined_fitness
else:
    print("Invalid choice! Defaulting to Crop Yield Fitness.")
    fitness_function = crop_yield_fitness

# Step 10: Genetic Algorithm with Temperature and Cooling Rate
def genetic_algorithm_with_temperature(fitness_function, generations=10):
    population = initialize_population(population_size)
    T = 100  # Initial temperature
    alpha = 0.95  # Cooling rate

    for generation in range(generations):
        # Sort population by fitness
        population = sorted(population, key=fitness_function, reverse=True)

        # Print the best solution in this generation
        best_solution = population[0]
        print(f"Generation {generation + 1} - Best Solution: {best_solution}, Fitness: {fitness_function(best_solution)}, Temperature: {T}")

        # Create new generation
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = select_parents(population, fitness_function)
            child1, child2 = crossover(parent1, parent2)

            # Mutate the children with temperature influence
            child1 = mutate_with_temperature(child1, T)
            child2 = mutate_with_temperature(child2, T)

            next_generation.append(child1)
            next_generation.append(child2)

        population = next_generation[:population_size]

        # Cool down the temperature
        T *= alpha

genetic_algorithm_with_temperature(fitness_function)


Select a fitness function:
1. Crop Yield Fitness
2. Environmental Impact Fitness
3. Combined Fitness
Enter 1, 2, or 3: 3
Generation 1 - Best Solution: ['Soybean', 'Summer', 'Phosphorus', 'None', 'Low', 'Medium', 'Organic'], Fitness: 48.23808614049371, Temperature: 100
Generation 2 - Best Solution: ['Soybean', 'Fall', 'Potassium', 'None', 'High', 'Medium', 'Organic'], Fitness: -20.74314457224926, Temperature: 95.0
Generation 3 - Best Solution: ['Wheat', 'Winter', 'Phosphorus', 'None', 'Low', 'Large', 'Organic'], Fitness: 58.83957705265945, Temperature: 90.25
Generation 4 - Best Solution: ['Wheat', 'Winter', 'Phosphorus', 'None', 'Low', 'Medium', 'Organic'], Fitness: -12.928541693298886, Temperature: 85.7375
Generation 5 - Best Solution: ['Rice', 'Fall', 'Potassium', 'None', 'High', 'Medium', 'Conventional'], Fitness: 51.7734619804165, Temperature: 81.45062499999999
Generation 6 - Best Solution: ['Rice', 'Fall', 'Potassium', 'None', 'High', 'Large', 'Conventional'], Fitness: 60.025696901

In [ ]:
# Step 1: Define possible values for genes
crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Step 2: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    return random.uniform(0, 50)

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 3: Generate random chromosome with added soil nutrients, farm size, and farm type
def generate_random_chromosome():
    # Limit fertilizer selection based on farm size
    farm_size = random.choice(farm_sizes)
    if farm_size == "Small":
        available_fertilizers = ["Organic", "Nitrogen"]
    elif farm_size == "Medium":
        available_fertilizers = fertilizers
    else:  # Large farms have all fertilizers
        available_fertilizers = fertilizers

    fertilizer = random.choice(available_fertilizers)

    # Limit pesticide selection based on farm type
    farm_type = random.choice(farm_types)
    if farm_type == "Organic":
        available_pesticides = ["None"]
    else:  # Conventional farms can use any pesticide
        available_pesticides = pesticides

    pesticide = random.choice(available_pesticides)

    # Limit soil nutrient selection based on farm size
    soil_nutrient = random.choice(soil_nutrients)

    return [
        random.choice(crop_types),
        random.choice(planting_times),
        fertilizer,
        pesticide,
        soil_nutrient,
        farm_size,
        farm_type
    ]

# Step 4: Initialize population
population_size = 10
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 5: Parent selection using fitness-based roulette selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)

    # Normalize fitness values to sum to 1
    normalized_fitness = [fitness / total_fitness for fitness in fitness_values]

    # Select two parents based on fitness probabilities
    parent1 = random.choices(population, weights=normalized_fitness, k=1)[0]
    parent2 = random.choices(population, weights=normalized_fitness, k=1)[0]

    return parent1, parent2

# Step 6: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 7: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        chromosome[mutation_point] = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        chromosome[mutation_point] = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        chromosome[mutation_point] = random.choice(farm_sizes)
    else:  # Farm type mutation
        chromosome[mutation_point] = random.choice(farm_types)

# Step 8: Mutation with temperature
def mutate_with_temperature(chromosome, temperature):
    mutation_point = random.randint(0, len(chromosome) - 1)

    # Check mutation points and adjust values based on limits
    if mutation_point == 0:
        new_value = random.choice(crop_types)
    elif mutation_point == 1:
        new_value = random.choice(planting_times)
    elif mutation_point == 2:  # Fertilizer mutation with limit based on farm size
        if chromosome[5] == "Small":
            available_fertilizers = ["Organic", "Nitrogen"]
        elif chromosome[5] == "Medium":
            available_fertilizers = fertilizers
        else:
            available_fertilizers = fertilizers
        new_value = random.choice(available_fertilizers)
    elif mutation_point == 3:  # Pesticide mutation with limit based on farm type
        if chromosome[6] == "Organic":
            available_pesticides = ["None"]
        else:
            available_pesticides = pesticides
        new_value = random.choice(available_pesticides)
    elif mutation_point == 4:  # Soil nutrient mutation with limit based on farm size
        new_value = random.choice(soil_nutrients)
    elif mutation_point == 5:  # Farm size mutation
        new_value = random.choice(farm_sizes)
    else:  # Farm type mutation
        new_value = random.choice(farm_types)

    # Simulated annealing-inspired acceptance
    original_fitness = combined_fitness(chromosome)
    chromosome[mutation_point] = new_value
    new_fitness = combined_fitness(chromosome)

    if new_fitness < original_fitness:
        acceptance_probability = random.uniform(0, 1)
        if acceptance_probability > (1 / (1 + temperature)):  # Revert mutation with a probability
            chromosome[mutation_point] = new_value

    return chromosome

# Step 9: User input for fitness function selection
print("Select a fitness function:")
print("1. Crop Yield Fitness")
print("2. Environmental Impact Fitness")
print("3. Combined Fitness")

user_choice = int(input("Enter 1, 2, or 3: "))

if user_choice == 1:
    fitness_function = crop_yield_fitness
elif user_choice == 2:
    fitness_function = environmental_impact_fitness
elif user_choice == 3:
    fitness_function = combined_fitness
else:
    print("Invalid choice! Defaulting to Crop Yield Fitness.")
    fitness_function = crop_yield_fitness

# Step 10: Genetic Algorithm with Temperature and Cooling Rate
def genetic_algorithm_with_temperature(fitness_function, generations=10):
    population = initialize_population(population_size)
    T = 100  # Initial temperature
    alpha = 0.95  # Cooling rate

    for generation in range(generations):
        # Sort population by fitness
        population = sorted(population, key=fitness_function, reverse=True)

        # Print the best solution in this generation
        best_solution = population[0]
        print(f"Generation {generation + 1} - Best Solution: {best_solution}, Fitness: {fitness_function(best_solution)}, Temperature: {T}")

        # Create new generation
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = select_parents(population, fitness_function)
            child1, child2 = crossover(parent1, parent2)

            # Mutate the children with temperature influence
            child1 = mutate_with_temperature(child1, T)
            child2 = mutate_with_temperature(child2, T)

            next_generation.append(child1)
            next_generation.append(child2)

        population = next_generation[:population_size]
        T *= alpha  # Cool down the temperature

    best_solution = max(population, key=fitness_function)
    return best_solution, fitness_function(best_solution)
    # ------------------------- NLP Component ------------------------------

def process_query():
    print("\nAnswer the following questions to optimize your farming decisions:")
    user_data = {}
    user_data['fertilizers_pesticides'] = input("1. What fertilizers and pesticides do you currently have access to? ")
    user_data['costs'] = input("2. What is the cost of each fertilizer and pesticide you use? ")
    user_data['eco_impact'] = input("3. Do you know the environmental impact or eco-scores of these products? (Yes/No): ")
    user_data['soil_nutrients'] = input("4. What are the nutrient requirements for your soil or crops? ")
    user_data['farm_size'] = input("5. What is the size of your farm (Small, Medium, Large)? ")

    print("\nBased on your answers, please choose a fitness function to optimize:")
    print("1. Maximize crop yield")
    print("2. Minimize environmental impact")
    print("3. Combined optimization (yield and impact)")
    choice = input("Enter 1, 2, or 3: ")

    if choice == "1":
        fitness_function = crop_yield_fitness
        explanation = "Maximizing crop yield."
    elif choice == "2":
        fitness_function = environmental_impact_fitness
        explanation = "Minimizing environmental impact."
    elif choice == "3":
        fitness_function = combined_fitness
        explanation = "Combining yield and environmental impact."
    else:
        print("Invalid choice. Defaulting to combined optimization.")
        fitness_function = combined_fitness
        explanation = "Default: Combining yield and environmental impact."

    return fitness_function, explanation, user_data

# Step 11: Run Genetic Algorithm with the selected fitness function
best_ga_solution, best_ga_fitness = genetic_algorithm_with_temperature(fitness_function)
print(f"\nBest solution using Genetic Algorithm: {best_ga_solution} with Fitness: {best_ga_fitness}")

Select a fitness function:
1. Crop Yield Fitness
2. Environmental Impact Fitness
3. Combined Fitness
Enter 1, 2, or 3: 3
Generation 1 - Best Solution: ['Corn', 'Winter', 'Nitrogen', 'None', 'Low', 'Large', 'Organic'], Fitness: 51.75875096725471, Temperature: 100
Generation 2 - Best Solution: ['Soybean', 'Summer', 'Nitrogen', 'Fungicide', 'High', 'Large', 'Conventional'], Fitness: 73.70174585592648, Temperature: 95.0
Generation 3 - Best Solution: ['Corn', 'Winter', 'Potassium', 'None', 'Medium', 'Small', 'Organic'], Fitness: 48.516929537286906, Temperature: 90.25
Generation 4 - Best Solution: ['Soybean', 'Summer', 'Nitrogen', 'None', 'High', 'Large', 'Conventional'], Fitness: 45.115258623735556, Temperature: 85.7375
Generation 5 - Best Solution: ['Soybean', 'Spring', 'Potassium', 'Insecticide', 'Low', 'Large', 'Conventional'], Fitness: -31.838520333621698, Temperature: 81.45062499999999
Generation 6 - Best Solution: ['Wheat', 'Summer', 'Potassium', 'None', 'Medium', 'Large', 'Convention

In [ ]:
import random

crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Step 1: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    return random.uniform(0, 50)

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 2: Generate random chromosome
def generate_random_chromosome():
    return [
        random.choice(crop_types),
        random.choice(planting_times),
        random.choice(fertilizers),
        random.choice(pesticides),
        random.choice(soil_nutrients),
        random.choice(farm_sizes),
        random.choice(farm_types),
    ]

# Step 3: Initialize population
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 4: Parent selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    parent1 = random.choices(population, weights=probabilities, k=1)[0]
    parent2 = random.choices(population, weights=probabilities, k=1)[0]
    return parent1, parent2

# Step 5: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 6: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:
        chromosome[mutation_point] = random.choice(fertilizers)
    elif mutation_point == 3:
        chromosome[mutation_point] = random.choice(pesticides)
    elif mutation_point == 4:
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:
        chromosome[mutation_point] = random.choice(farm_sizes)
    elif mutation_point == 6:
        chromosome[mutation_point] = random.choice(farm_types)
    return chromosome



#Genetic Algo

def genetic_algorithm(fitness_function, population_size=10, generations=20):
    population = initialize_population(population_size)
    for _ in range(generations):
        population = sorted(population, key=fitness_function, reverse=True)
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = select_parents(population, fitness_function)
            child1, child2 = crossover(parent1, parent2)
            next_generation.extend([mutate(child1), mutate(child2)])
        population = next_generation[:popuation_size]
    return max(population, key=fitness_function)


    #NLP Component

def process_query():
    print("\nAnswer the following questions to optimize your farming decisions:")
    user_data = {}
    user_data['fertilizers_pesticides'] = input("1. What fertilizers and pesticides do you currently have access to? ")
    user_data['costs'] = input("2. What is the cost of each fertilizer and pesticide you use? ")
    user_data['eco_impact'] = input("3. Do you know the environmental impact or eco-scores of these products? (Yes/No): ")
    user_data['soil_nutrients'] = input("4. What are the nutrient requirements for your soil or crops? ")
    user_data['farm_size'] = input("5. What is the size of your farm (Small, Medium, Large)? ")


    print("\nBased on your answers, please choose a fitness function to optimize:")
    print("1. Maximize crop yield")
    print("2. Minimize environmental impact")
    print("3. Combined optimization (yield and impact)")
    choice = input("Enter 1, 2, or 3: ")

    if choice == "1":
        fitness_function = crop_yield_fitness
        explanation = "Maximizing crop yield."
    elif choice == "2":
        fitness_function = environmental_impact_fitness
        explanation = "Minimizing environmental impact."
    elif choice == "3":
        fitness_function = combined_fitness
        explanation = "Combining yield and environmental impact."
    else:
        print("Invalid choice. Defaulting to combined optimization.")
        fitness_function = combined_fitness
        explanation = "Default: Combining yield and environmental impact."

    return fitness_function, explanation, user_data

print("Hello!!!Intelligent Smart farm Optimization System!")
fitness_function, explanation, user_data = process_query()
# Step 11: Run Genetic Algorithm with the selected fitness function
best_ga_solution, best_ga_fitness = genetic_algorithm_with_temperature(fitness_function)
print(f"\nBest solution using Genetic Algorithm: {best_ga_solution} with Fitness: {best_ga_fitness}")



Hello!!!Intelligent Smart farm Optimization System!

Answer the following questions to optimize your farming decisions:
1. What fertilizers and pesticides do you currently have access to? nitro
2. What is the cost of each fertilizer and pesticide you use? 2
3. Do you know the environmental impact or eco-scores of these products? (Yes/No): no
4. What are the nutrient requirements for your soil or crops? soil
5. What is the size of your farm (Small, Medium, Large)? small

Based on your answers, please choose a fitness function to optimize:
1. Maximize crop yield
2. Minimize environmental impact
3. Combined optimization (yield and impact)
Enter 1, 2, or 3: 1
Generation 1 - Best Solution: ['Corn', 'Winter', 'Phosphorus', 'None', 'Low', 'Large', 'Organic'], Fitness: 82.36613224293549, Temperature: 100
Generation 2 - Best Solution: ['Soybean', 'Fall', 'Potassium', 'Fungicide', 'Medium', 'Large', 'Organic'], Fitness: 50.504192021683494, Temperature: 95.0
Generation 3 - Best Solution: ['Corn',

In [ ]:
import random

crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Step 1: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    return random.uniform(0, 50)

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 2: Generate random chromosome
def generate_random_chromosome():
    return [
        random.choice(crop_types),
        random.choice(planting_times),
        random.choice(fertilizers),
        random.choice(pesticides),
        random.choice(soil_nutrients),
        random.choice(farm_sizes),
        random.choice(farm_types),
    ]

# Step 3: Initialize population
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 4: Parent selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    parent1 = random.choices(population, weights=probabilities, k=1)[0]
    parent2 = random.choices(population, weights=probabilities, k=1)[0]
    return parent1, parent2

# Step 5: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 6: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:
        chromosome[mutation_point] = random.choice(fertilizers)
    elif mutation_point == 3:
        chromosome[mutation_point] = random.choice(pesticides)
    elif mutation_point == 4:
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:
        chromosome[mutation_point] = random.choice(farm_sizes)
    elif mutation_point == 6:
        chromosome[mutation_point] = random.choice(farm_types)
    return chromosome



#Genetic Algo

def genetic_algorithm(fitness_function, population_size=10, generations=20):
    population = initialize_population(population_size)
    for _ in range(generations):
        population = sorted(population, key=fitness_function, reverse=True)
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = select_parents(population, fitness_function)
            child1, child2 = crossover(parent1, parent2)
            next_generation.extend([mutate(child1), mutate(child2)])
        population = next_generation[:popuation_size]
    return max(population, key=fitness_function)


    #NLP Component

def process_query():
    print("\nAnswer the following questions to optimize your farming decisions:")
    user_data = {}
    user_data['fertilizers_pesticides'] = input("1. What fertilizers and pesticides do you currently have access to? ")
    user_data['costs'] = input("2. What is the cost of each fertilizer and pesticide you use? ")
    user_data['eco_impact'] = input("3. Do you know the environmental impact or eco-scores of these products? (Yes/No): ")
    user_data['soil_nutrients'] = input("4. What are the nutrient requirements for your soil or crops? ")
    user_data['farm_size'] = input("5. What is the size of your farm (Small, Medium, Large)? ")


    print("\nBased on your answers, please choose a fitness function to optimize:")
    print("1. Maximize crop yield")
    print("2. Minimize environmental impact")
    print("3. Combined optimization (yield and impact)")
    choice = input("Enter 1, 2, or 3: ")

    if choice == "1":
        fitness_function = crop_yield_fitness
        explanation = "Maximizing crop yield."
    elif choice == "2":
        fitness_function = environmental_impact_fitness
        explanation = "Minimizing environmental impact."
    elif choice == "3":
        fitness_function = combined_fitness
        explanation = "Combining yield and environmental impact."
    else:
        print("Invalid choice. Defaulting to combined optimization.")
        fitness_function = combined_fitness
        explanation = "Default: Combining yield and environmental impact."

    return fitness_function, explanation, user_data

print("Hello!!!Intelligent Smart farm Optimization System!")
fitness_function, explanation, user_data = process_query()
# Step 11: Run Genetic Algorithm with the selected fitness function
best_ga_solution, best_ga_fitness = genetic_algorithm_with_temperature(fitness_function)
print(f"\nBest solution using Genetic Algorithm: {best_ga_solution} with Fitness: {best_ga_fitness}")



Hello!!!Intelligent Smart farm Optimization System!

Answer the following questions to optimize your farming decisions:
1. What fertilizers and pesticides do you currently have access to? nitro
2. What is the cost of each fertilizer and pesticide you use? 2
3. Do you know the environmental impact or eco-scores of these products? (Yes/No): no
4. What are the nutrient requirements for your soil or crops? soil
5. What is the size of your farm (Small, Medium, Large)? small

Based on your answers, please choose a fitness function to optimize:
1. Maximize crop yield
2. Minimize environmental impact
3. Combined optimization (yield and impact)
Enter 1, 2, or 3: 1
Generation 1 - Best Solution: ['Corn', 'Winter', 'Phosphorus', 'None', 'Low', 'Large', 'Organic'], Fitness: 82.36613224293549, Temperature: 100
Generation 2 - Best Solution: ['Soybean', 'Fall', 'Potassium', 'Fungicide', 'Medium', 'Large', 'Organic'], Fitness: 50.504192021683494, Temperature: 95.0
Generation 3 - Best Solution: ['Corn',

In [ ]:
import random

crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Step 1: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    return random.uniform(0, 50)

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 2: Generate random chromosome
def generate_random_chromosome():
    return [
        random.choice(crop_types),
        random.choice(planting_times),
        random.choice(fertilizers),
        random.choice(pesticides),
        random.choice(soil_nutrients),
        random.choice(farm_sizes),
        random.choice(farm_types),
    ]

# Step 3: Initialize population
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 4: Parent selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    parent1 = random.choices(population, weights=probabilities, k=1)[0]
    parent2 = random.choices(population, weights=probabilities, k=1)[0]
    return parent1, parent2

# Step 5: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 6: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:
        chromosome[mutation_point] = random.choice(fertilizers)
    elif mutation_point == 3:
        chromosome[mutation_point] = random.choice(pesticides)
    elif mutation_point == 4:
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:
        chromosome[mutation_point] = random.choice(farm_sizes)
    elif mutation_point == 6:
        chromosome[mutation_point] = random.choice(farm_types)
    return chromosome



#Genetic Algo

def genetic_algorithm(fitness_function, population_size=10, generations=20):
    population = initialize_population(population_size)
    for _ in range(generations):
        population = sorted(population, key=fitness_function, reverse=True)
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = select_parents(population, fitness_function)
            child1, child2 = crossover(parent1, parent2)
            next_generation.extend([mutate(child1), mutate(child2)])
        population = next_generation[:popuation_size]
    return max(population, key=fitness_function)


    #NLP Component

def process_query():
    print("\nAnswer the following questions to optimize your farming decisions:")
    user_data = {}
    user_data['fertilizers_pesticides'] = input("1. What fertilizers and pesticides do you currently have access to? ")
    user_data['costs'] = input("2. What is the cost of each fertilizer and pesticide you use? ")
    user_data['eco_impact'] = input("3. Do you know the environmental impact or eco-scores of these products? (Yes/No): ")
    user_data['soil_nutrients'] = input("4. What are the nutrient requirements for your soil or crops? ")
    user_data['farm_size'] = input("5. What is the size of your farm (Small, Medium, Large)? ")
    user_data['farm_types']= input("6.whats the types of your farm(Organic,Conventional)? ")

    print("\nBased on your answers, please choose a fitness function to optimize:")
    print("1. Maximize crop yield")
    print("2. Minimize environmental impact")
    print("3. Combined optimization (yield and impact)")
    choice = input("Enter 1, 2, or 3: ")

    if choice == "1":
        fitness_function = crop_yield_fitness
        explanation = "Maximizing crop yield."
    elif choice == "2":
        fitness_function = environmental_impact_fitness
        explanation = "Minimizing environmental impact."
    elif choice == "3":
        fitness_function = combined_fitness
        explanation = "Combining yield and environmental impact."
    else:
        print("Invalid choice. Defaulting to combined optimization.")
        fitness_function = combined_fitness
        explanation = "Default: Combining yield and environmental impact."

    return fitness_function, explanation, user_data

print("Hello!!!Intelligent Smart farm Optimization System!")
fitness_function, explanation, user_data = process_query()
# Step 11: Run Genetic Algorithm with the selected fitness function
best_ga_solution, best_ga_fitness = genetic_algorithm_with_temperature(fitness_function)
print(f"\nBest solution using Genetic Algorithm: {best_ga_solution} with Fitness: {best_ga_fitness}")



Hello!!!Intelligent Smart farm Optimization System!

Answer the following questions to optimize your farming decisions:
1. What fertilizers and pesticides do you currently have access to? nitrogen
2. What is the cost of each fertilizer and pesticide you use? 10
3. Do you know the environmental impact or eco-scores of these products? (Yes/No): no
4. What are the nutrient requirements for your soil or crops? small
5. What is the size of your farm (Small, Medium, Large)? small
6.whats the types of your farm(Organic,Conventional)? organic

Based on your answers, please choose a fitness function to optimize:
1. Maximize crop yield
2. Minimize environmental impact
3. Combined optimization (yield and impact)
Enter 1, 2, or 3: 2
Generation 1 - Best Solution: ['Soybean', 'Spring', 'Potassium', 'Insecticide', 'High', 'Medium', 'Conventional'], Fitness: 22.583325987597185, Temperature: 100
Generation 2 - Best Solution: ['Corn', 'Summer', 'Organic', 'None', 'Low', 'Small', 'Organic'], Fitness: 31.

In [ ]:
import random

crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Step 1: Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(0, 100)

def environmental_impact_fitness(chromosome):
    return random.uniform(0, 50)

def combined_fitness(chromosome):
    return crop_yield_fitness(chromosome) - environmental_impact_fitness(chromosome)

# Step 2: Generate random chromosome
def generate_random_chromosome():
    return [
        random.choice(crop_types),
        random.choice(planting_times),
        random.choice(fertilizers),
        random.choice(pesticides),
        random.choice(soil_nutrients),
        random.choice(farm_sizes),
        random.choice(farm_types),
    ]

# Step 3: Initialize population
def initialize_population(size):
    return [generate_random_chromosome() for _ in range(size)]

# Step 4: Parent selection
def select_parents(population, fitness_function):
    fitness_values = [fitness_function(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    parent1 = random.choices(population, weights=probabilities, k=1)[0]
    parent2 = random.choices(population, weights=probabilities, k=1)[0]
    return parent1, parent2

# Step 5: Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return (
        parent1[:point] + parent2[point:],
        parent2[:point] + parent1[point:],
    )

# Step 6: Mutation
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)
    if mutation_point == 0:
        chromosome[mutation_point] = random.choice(crop_types)
    elif mutation_point == 1:
        chromosome[mutation_point] = random.choice(planting_times)
    elif mutation_point == 2:
        chromosome[mutation_point] = random.choice(fertilizers)
    elif mutation_point == 3:
        chromosome[mutation_point] = random.choice(pesticides)
    elif mutation_point == 4:
        chromosome[mutation_point] = random.choice(soil_nutrients)
    elif mutation_point == 5:
        chromosome[mutation_point] = random.choice(farm_sizes)
    elif mutation_point == 6:
        chromosome[mutation_point] = random.choice(farm_types)
    return chromosome



#Genetic Algo

def genetic_algorithm(fitness_function, population_size=10, generations=20):
    population = initialize_population(population_size)
    for _ in range(generations):
        population = sorted(population, key=fitness_function, reverse=True)
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = select_parents(population, fitness_function)
            child1, child2 = crossover(parent1, parent2)
            next_generation.extend([mutate(child1), mutate(child2)])
        population = next_generation[:popuation_size]
    return max(population, key=fitness_function)


    #NLP Component

def process_query():
    print("\nAnswer the following questions to optimize your farming decisions:")
    user_data = {}
    user_data['fertilizers_pesticides'] = input("1. What fertilizers and pesticides do you currently have access to? ")
    user_data['costs'] = input("2. What is the cost of each fertilizer and pesticide you use? ")
    user_data['eco_impact'] = input("3. Do you know the environmental impact or eco-scores of these products? (Yes/No): ")
    user_data['soil_nutrients'] = input("4. What are the nutrient requirements for your soil or crops? ")
    user_data['farm_size'] = input("5. What is the size of your farm (Small, Medium, Large)? ")
    user_data['farm_types']= input("6.whats the types of your farm(Organic,Conventional)? ")

    print("\nBased on your answers, please choose a fitness function to optimize:")
    print("1. Maximize crop yield")
    print("2. Minimize environmental impact")
    print("3. Combined optimization (yield and impact)")
    choice = input("Enter 1, 2, or 3: ")

    if choice == "1":
        fitness_function = crop_yield_fitness
        explanation = "Maximizing crop yield."
    elif choice == "2":
        fitness_function = environmental_impact_fitness
        explanation = "Minimizing environmental impact."
    elif choice == "3":
        fitness_function = combined_fitness
        explanation = "Combining yield and environmental impact."
    else:
        print("Invalid choice. Defaulting to combined optimization.")
        fitness_function = combined_fitness
        explanation = "Default: Combining yield and environmental impact."

    return fitness_function, explanation, user_data

print("Hello!!!Intelligent Smart farm Optimization System!")
fitness_function, explanation, user_data = process_query()
# Step 11: Run Genetic Algorithm with the selected fitness function
best_ga_solution, best_ga_fitness = genetic_algorithm_with_temperature(fitness_function)
print(f"\nBest solution using Genetic Algorithm: {best_ga_solution} with Fitness: {best_ga_fitness}")



Hello!!!Intelligent Smart farm Optimization System!

Answer the following questions to optimize your farming decisions:
